# Recorded Demo of RecNextEval for SIGIR 2026 Presentation

Paper title: RecNextEval: A Reference Implementation for Temporal Next-Batch Recommendation Evaluation

Presenter: Ng Tze Kean

Presentation paramters:

- Dataset: MovieLens100K
- Top K = 10
- Algorithm: ItemKNNIncremental, RecentPopularity
- First timestamp split at epoch time 875_156_710

In [ ]:
from datetime import datetime, timezone


epoch_ts = 875_156_710
converted_dt = datetime.fromtimestamp(epoch_ts, tz=timezone.utc)
converted_dt

In [ ]:
from recnexteval.datasets import MovieLens100K
from recnexteval.settings import SlidingWindowSetting


k = 10
dataset = MovieLens100K()
data = dataset.load()

setting_window = SlidingWindowSetting(
    training_t=875_156_710,
    window_size=60 * 60 * 24 * 30,  # day times N
    top_K=k
)

setting_window.split(data)

## Create evaluator instance

In [ ]:
from recnexteval.evaluators import EvaluatorStreamerBuilder


builder = EvaluatorStreamerBuilder(
    ignore_unknown_item=False,
    ignore_unknown_user=False,
)
builder.add_setting(setting=setting_window)
builder.set_metric_k(k)
builder.add_metric("HitK")
builder.add_metric("NDCGK")
evaluator = builder.build()

In [ ]:
from recnexteval.algorithms import ItemKNNIncremental, RecentPopularity


external_model = [ItemKNNIncremental(K=k), RecentPopularity(K=k)]
ids = []
for model in external_model:
    ids.append(evaluator.register_model(algorithm=model))
print(ids)

In [ ]:
evaluator.start_stream()

## Demo Steps

### Data release phase

In [ ]:
for model, model_id in zip(external_model, ids):
    training_data = evaluator.get_training_data(model_id)
    if training_data is None:
        raise ValueError("No data available for the external model.")
    model.fit(training_data)

### Prediction phase

In [ ]:
for model, model_id in zip(external_model, ids):
    unlabeled_data = evaluator.get_unlabeled_data(model_id)
    prediction = model.predict(unlabeled_data)
    evaluator.submit_prediction(model_id, prediction)

## Iterate through windows

Evaluator iterates through time windows, releasing data and collecting predictions at each step.

In [ ]:
for i in range(setting_window.num_split - 1):
    for model, model_id in zip(external_model, ids):
        training_data = evaluator.get_training_data(model_id)
        model.fit(training_data)
        unlabeled_data = evaluator.get_unlabeled_data(model_id)
        prediction = model.predict(unlabeled_data)
        evaluator.submit_prediction(model_id, prediction)

## Evaluate model performance

In [ ]:
evaluator.get_all_algorithm_status()

In [ ]:
evaluator.metric_results("macro")

In [ ]:
evaluator.metric_results("micro")


In [ ]:
evaluator.metric_results("user")

In [ ]:
evaluator.metric_results("window")

In [ ]:
evaluator.plot_window_level_metric()